## SBI

In [4]:
import os
import re
import pandas as pd
from PyPDF2 import PdfReader

In [5]:
def process_pdf_text(pdf_path):
    """
    Function to extract text from all pages of a given PDF.
    """
    reader = PdfReader(pdf_path)
    num_pages = len(reader.pages)
    combined_text = ""
    
    for i in range(num_pages):
        page = reader.pages[i]
        text = page.extract_text()
        if text:
            combined_text += text
    
    return combined_text



def date_removed(b2):
    date_pattern = r"\b\d{2}/\d{2}/\d{4}\b"
    filtered_list = [item for item in b2 if not re.match(date_pattern, item)]
    return filtered_list


def removed_change(list_of_lists):
    # Use list comprehension to filter out sublists containing the word "Levied"
    filtered_list = [sublist for sublist in list_of_lists if not any("***Change of" in str(item) for item in sublist)]
    return filtered_list


def removed_invalid(list_of_lists):
    # Use list comprehension to filter out sublists containing the word "Levied"
    filtered_list = [sublist for sublist in list_of_lists if not any("***Invalid" in str(item) for item in sublist)]
    return filtered_list


def com_name(c2):
    com =[]
    for i in c2:
        j = str(i).split("\n")
        com.append(j[0])
    return com



import re

def extract_values_from_date_regex(data_list):
    # Combine the list into a single string with spaces between elements
    data_str = ' '.join(data_list)
    
    # Define a regex pattern for the date (DD/MM/YYYY) and the next 5 values
    # pattern = r'(\d{2}/\d{2}/\d{4}(?:\s\S+){5})'

    # pattern = r"(\b\d{2}-[A-Za-z]{3}-\d{4}(?:\s\S+){5})"

    pattern = r'(\b\d{2}-(?:[A-Za-z]{3})-\d{4}\b|\b\d{2}/\d{2}/\d{4}\b)(?:\s+(\S+)){5}'
    
    # Find all matches for the pattern in the string
    matches = re.finditer(pattern, data_str)
    
    # Extract the values from the matches
    extracted_values = []
    for match in matches:
        values = match.group()  # Get the whole match as a string
        values_list = values.split()  # Convert the string back to a list
        extracted_values.append(values_list)
    
    return extracted_values



def removed_levied(list_of_lists):
    # Use list comprehension to filter out sublists containing the word "Levied"
    filtered_list = [sublist for sublist in list_of_lists if not any("Levied" in str(item) for item in sublist)]
    return filtered_list


def process_exit_load1(list_of_lists):
    # Use list comprehension to filter out sublists containing the word "Levied"
    filtered_list = [sublist for sublist in list_of_lists if not any("Exit Load" in str(item) for item in sublist)]
    return filtered_list



def process_e1(a):
    e1 = []
    
    for sublist in a:
        for item in sublist:
            c = str(item).replace("(", "").replace(")", "").replace(",", "").split("\n")
            e1.append(c)
    
    return e1



def transaction_type(removed_items):
    t = []
    for i in removed_items:
        for j in i:
            if j.lower().startswith("redemption") :
                t.append(j)
            if j.lower().startswith("purchase"):
                t.append(j)
    return t


def balance_unit_pop(e):
    i1 = []
    for i in e:
        if len(i) ==2:
            i1.append(i.pop(0))
    return e


def process_exit_load(data):
    removed_items = []
    for sublist in data:
        indices_to_remove = []
        for i in range(1, len(sublist)):
            if sublist[i].startswith('*** Exit Load Deducted') :
                indices_to_remove.append(i - 1)
        # Remove items in reverse order to avoid index shifting issues
        for index in sorted(indices_to_remove, reverse=True):
            removed_items.append(sublist[index])
            del sublist[index]
    return data


def cleaned_data(removed_items):

    b1 =[]
    for i in removed_items:
        for j in i:
            if j[:1].isdigit():
                b1.append(j)
    return b1




def insi_number_extract(pdf_path):
    reader = PdfReader(pdf_path)
    isin_pattern = r"INF"
    extracted_data = []
    num_pages = len(reader.pages)
    for i in range(num_pages):
        page = reader.pages[i]
        text = page.extract_text()
        tes = text.replace(',', '').replace("(", "").replace(')', '').replace("NAV", "")
        indices = [m.start(0) for m in re.finditer(isin_pattern, tes)]
        
        for i in indices:
            vv = tes[i:]
            if "RedemptionElectronic" in vv or "Purchase DT" in vv or "RedemptionRTGS" in vv:
                vb = tes[i:i+13]
                vvv = vb.replace('\n', "")
                extracted_data.append(vvv)
                
    return extracted_data

In [6]:
# all_groups = []
# com_nam = []
# folio_list = []
# folio = []

# folio_pattern = r'Folio Number\s*:\s*(\d+(?:\s*/\s*\d+)?)'
# # folio_numbers_pattern = r'Folio Number\s*:\s*(\d+(?:\s*/\s*\d+)?)'


# # Loop through each Excel file
# for excel_file in os.listdir(excel_folder):
#     xl_path = os.path.join(excel_folder, excel_file)
#     print(f"Processing Excel: {xl_path}")
    
#     if xl_path.endswith(".xlsx"):
#         xls = pd.ExcelFile(xl_path)
#         table_names = xls.sheet_names
        
#         df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
#         a1 = df1.values.tolist()
#         c2 = []
#         c = a1[0:2][1][0]
#         c2.append(c)
#         com_na = com_name(c2)
#         print(com_na)
#         com_nam.extend(com_na)
        
#         all_tran = []

#         for sheet in table_names:
#             df = pd.read_excel(xls, sheet_name=sheet, header=None)
#             a = df.values.tolist()
#             a1 = removed_invalid(a)
#             cl = removed_levied(a1)
#             ch = removed_change(cl)
#             re1 = process_exit_load1(ch)
#             e = process_e1(re1)
#             u = transaction_type(e) 
#             b = balance_unit_pop(e)
#             removed_items = process_exit_load(b)
#             b2 = cleaned_data(removed_items)
#             v = date_removed(b2)
           
#             all_tran.extend(u)
#             all_tran1 = [item for item in all_tran if item != 'Redemption']

#             if len(u) !=0:
#                 groups = extract_values_from_date_regex(v)
#                 all_groups.extend(groups)
            
            
#         df = pd.DataFrame(all_groups)

#         if df.shape[1] > 2:
#             corresponding_pdf_file = excel_file.replace('.xlsx', '.pdf')
#             pdf_path = os.path.join(pdf_file_folder, corresponding_pdf_file)
#             print(pdf_path)
#             if os.path.exists(pdf_path):
#                 reader = PdfReader(pdf_path)
#                 page = reader.pages[0]
#                 text = page.extract_text()
    
#                 numbers = re.findall(folio_pattern, text)
#                 folio.append(numbers)

#                 # print(numbers)
       
#             df.columns = ['Date','Amount(INR)', 'NAV(INR)', 'Price(INR)', 'No_of_Unit', 'Balance_Unit']
#             df['Company Name'] = pd.Series([com_nam[0]] * len(df), index=df.index) 
#             df['Transaction Type'] = pd.Series(all_tran1)
#             df['Folio Number'] = pd.Series([folio[0][0]] * len(df), index=df.index)

#             print(df)

#             all_groups.clear()
#             com_nam.clear()
#             all_tran1.clear()
#             folio.pop(0)


In [9]:

output_folder = r"D:\Python\ajit\SBI\final_csv"

excel_folder = r'D:\Python\ajit\SBI\pdf_to_excel'

pdf_file_folder = r"D:\Python\ajit\SBI\SBI PDF"

In [10]:

isin_pattern = r"INF"
all_groups = []
com_nam = []
folio = []
len_u = []
inf = []
folio_pattern = r'Folio Number\s*:\s*(\d+(?:\s*/\s*\d+)?)'

# Loop through each Excel file
for excel_file in os.listdir(excel_folder):
    xl_path = os.path.join(excel_folder, excel_file)
    print(f"Processing Excel: {xl_path}")
    
    if xl_path.endswith(".xlsx"):
        xls = pd.ExcelFile(xl_path)
        table_names = xls.sheet_names
        
        df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
        a1 = df1.values.tolist()
        c2 = []
        c = a1[0:2][1][0]
        c2.append(c)
        com_na = com_name(c2)
        print(com_na)
        com_nam.extend(com_na)
        
        all_tran = []

        for sheet in table_names:
            df = pd.read_excel(xls, sheet_name=sheet, header=None)
            a = df.values.tolist()
            a1 = removed_invalid(a)
            cl = removed_levied(a1)
            ch = removed_change(cl)
            re1 = process_exit_load1(ch)
            e = process_e1(re1)
            u = transaction_type(e)

            b = balance_unit_pop(e)
            removed_items = process_exit_load(b)
            b2 = cleaned_data(removed_items)
            v = date_removed(b2)
           
            all_tran.extend(u)
            all_tran1 = [item for item in all_tran if item != 'Redemption']

            if len(u) !=0:
                groups = extract_values_from_date_regex(v)
                le = len(groups)
                len_u.append(le)
                all_groups.extend(groups)
                   
            
        df = pd.DataFrame(all_groups)
        # print(df.shape[1])
        if df.shape[1] == 0:
            com_nam.clear()

        else:
            if df.shape[1] > 2:
                corresponding_pdf_file = excel_file.replace('.xlsx', '.pdf')
                pdf_path = os.path.join(pdf_file_folder, corresponding_pdf_file)
                if os.path.exists(pdf_path):
                    reader = PdfReader(pdf_path)
                    page = reader.pages[0]
                    text = page.extract_text()
        
                    numbers = re.findall(folio_pattern, text)
                    folio.append(numbers)
                    extracted_info = insi_number_extract(pdf_path)
                    inf.extend(extracted_info)

                    print(extracted_info,'======================')

            
                kk = [item for item in len_u if item != 0]
                result = [s for s, n in zip(inf, kk) for _ in range(n)]
                df.columns = ['Date','Amount(INR)', 'NAV(INR)', 'Price(INR)', 'No_of_Unit', 'Balance_Unit']
                df['Company Name'] = pd.Series([com_nam[0]] * len(df), index=df.index) 
                df['Transaction Type'] = pd.Series(all_tran1)
                df['Folio Number'] = pd.Series([folio[0][0]] * len(df), index=df.index)
                df['ISIN Number'] = pd.Series(result)

                print(df)

                # Save the DataFrame to CSV
                csv_filename = f"{os.path.splitext(excel_file)[0]}.csv"
                csv_path = os.path.join(output_folder, csv_filename)
                df.to_csv(csv_path, index=False)
                print(f"Data saved to {csv_path}")

                all_groups.clear()
                com_nam.clear()
                all_tran1.clear()
                folio.pop(0)
                inf.clear()
                len_u.clear()


Processing Excel: D:\Python\ajit\SBI\pdf_to_excel\Acctstmt_L_13317758 - 2024-08-31T101247.357.xlsx
['Adani Power Limited']
D:\Python\ajit\SBI\SBI PDF\Acctstmt_L_13317758 - 2024-08-31T101247.357.pdf
['INF200K01TK5 ', 'INF200K01TK5 ', 'INF200K01TK5 ', 'INF200K01UT4 '] ======================
           Date     Amount(INR)   NAV(INR) Price(INR)   No_of_Unit  \
0   04-Aug-2024   3000000000.00  3983.5981  3983.5981   753088.019   
1   05-Aug-2024    699965001.75  3983.5981  3983.5981   175711.752   
2   11-Aug-2024   5922821497.64  3988.4210  3988.4210  1485004.090   
3   13-Aug-2024   2229888505.57  3989.1048  3989.1048   558994.716   
4   14-Aug-2024    349982500.87  3989.7951  3989.7951    87719.417   
5   14-Aug-2024     69996500.17  3989.7951  3989.7951    17543.883   
6   15-Aug-2024   2000000000.00  3991.1818  3991.1818   501104.710   
7   16-Aug-2024   1999900005.00  3991.1818  3991.1818   501079.656   
8   19-Aug-2024   1550000000.00  3993.9555  3993.9555   388086.447   
9   20-Aug